In [1]:
! python -m pip install python-dotenv

In [2]:
import logging
import boto3
import os
import io
from botocore.config import Config
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_columns", None)
from dotenv import load_dotenv
load_dotenv('/home/train/mlops4/02_mlops_docker/.env')

True

# With keys

In [3]:
ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

In [4]:
print(ACCESS_KEY)
print(SECRET_KEY)

trainkey
trainsecret


In [18]:
def get_s3_client():
    s3 = boto3.client('s3',
                      endpoint_url='http://localhost:9000',
                      aws_access_key_id='trainkey',
                      aws_secret_access_key='trainsecret',
                      config=Config(signature_version='s3v4'))
    return s3

In [6]:
def load_df_from_s3(bucket, key, sep=",", index_col=None, usecols=None):
    ''' Read a csv from a s3 bucket & load into pandas dataframe'''
    s3 = get_s3_client()
    try:
        logging.info(f"Loading {bucket, key}")
        obj = s3.get_object(Bucket=bucket, Key=key)
        return pd.read_csv(obj['Body'], sep=sep, index_col=index_col, usecols=usecols, low_memory=False)
    except botocore.exceptions.ClientError as err:
        status = err.response["ResponseMetadata"]["HTTPStatusCode"]
        errcode = err.response["Error"]["Code"]
        if status == 404:
            logging.warning("Missing object, %s", errcode)
        elif status == 403:
            logging.error("Access denied, %s", errcode)
        else:
            logging.exception("Error in request, %s", errcode)

In [7]:
def save_df_to_s3(df, bucket, key):
    ''' Store file to s3'''
    s3 = get_s3_client()
    try:
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
        logging.info(f"{key} saved to s3 bucket {bucket}")
    except Exception as e:
        raise logging.exception(e)

In [8]:
! mkdir  ~/datasets/

In [9]:
! curl -o ~/datasets/Advertising.csv https://raw.githubusercontent.com/erkansirin78/datasets/master/Advertising.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4556  100  4556    0     0   9432      0 --:--:-- --:--:-- --:--:--  9413


In [10]:
df = pd.read_csv("/home/train/datasets/Advertising.csv")

In [11]:
df.head()

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Save to S3 (MinIO)

## Create bucket

In [19]:
s3 = get_s3_client()

In [20]:
s3.create_bucket(Bucket="mlops-test")

{'ResponseMetadata': {'RequestId': '1806B228F70E1361',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'location': '/mlops-test',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '1806B228F70E1361',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '1126',
   'x-ratelimit-remaining': '1126',
   'x-xss-protection': '1; mode=block',
   'date': 'Sun, 10 Nov 2024 19:29:55 GMT'},
  'RetryAttempts': 0},
 'Location': '/mlops-test'}

In [21]:
save_df_to_s3(df=df, bucket='mlops-test', key='Advertising.csv')

# Read from S3 (MinIO)

In [22]:
df_from_s3 = load_df_from_s3( bucket='mlops-test', key='Advertising.csv')

In [23]:
df_from_s3.head()

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
